# Prediction of winning probability of team

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
df=pd.read_csv("all_season_details.csv")
df2=pd.read_csv("all_season_summary.csv")
df3=pd.read_csv("all_season_batting_card.csv")

In [3]:
df2.rename(columns={"id":"match_id"},inplace=True)  # To match with df1

In [4]:
df=df.merge(df2[["match_id","winner"]])

In [5]:
df=df.apply(lambda x: x.replace("KXIP","PBKS"))  # KXIP was changed to PBKS
df2=df2.apply(lambda x: x.replace("KXIP","PBKS"))

In [6]:
# Dropping the old teams
df.drop(df[(df["home_team"]=="RPS") | (df["away_team"]=="RPS")].index,axis=0,inplace=True)
df.drop(df[(df["home_team"]=="GL") | (df["away_team"]=="GL")].index,axis=0,inplace=True)
df.drop(df[(df["home_team"]=="Kochi") | (df["away_team"]=="Kochi")].index,axis=0,inplace=True)
df.drop(df[(df["home_team"]=="PWI") | (df["away_team"]=="PWI")].index,axis=0,inplace=True)

df2.drop(df2[(df2["home_team"]=="RPS") | (df2["away_team"]=="RPS")].index,axis=0,inplace=True)
df2.drop(df2[(df2["home_team"]=="GL") | (df2["away_team"]=="GL")].index,axis=0,inplace=True)
df2.drop(df2[(df2["home_team"]=="Kochi") | (df2["away_team"]=="Kochi")].index,axis=0,inplace=True)
df2.drop(df2[(df2["home_team"]=="PWI") | (df2["away_team"]=="PWI")].index,axis=0,inplace=True)

In [7]:
# List of match ids which are not comon in both df1 and df2
#list(pd.Series(df2["match_id"].unique())[~pd.Series(df2["match_id"].unique()).isin(list(df["match_id"].unique()))])  

In [8]:
df2.drop(df2[df2["match_id"].isin(list(pd.Series(df2["match_id"].unique())[~pd.Series(df2["match_id"].unique()).isin(list(df["match_id"].unique()))]))].index,axis=0,inplace=True)  # Removing matches from df2 which are not in common in both df1 and df2
df2.reset_index(inplace=True,drop=True) 

In [9]:
#df2["2nd_inning_score"].isnull().sum()

In [10]:
df2.dropna(subset=["2nd_inning_score"],inplace=True,axis=0) #Removing null row from 2nd_inning_score feature
df2.reset_index(inplace=True,drop=True)

In [11]:
df.drop(df[df["match_id"].isin([829763,501265])].index,inplace=True,axis=0) # Removing matches from df which are not in common in both df1 and df2
df.reset_index(inplace=True,drop=True)

In [12]:
# Extracting score from the features
df2["1st_inning_score"]=df2["1st_inning_score"].apply(lambda x: x.split("/")[0])
df2["2nd_inning_score"]=df2["2nd_inning_score"].apply(lambda x: x.split("/")[0])

In [13]:
df=df.merge(df2[["match_id","1st_inning_score"]])
df=df.merge(df2[["match_id","2nd_inning_score"]])

In [14]:
batting_team=[]
bowling_team=[]

In [15]:
df2.shape

(834, 45)

In [16]:
# Creating new features batting_team and bowling_team
for i in range(df2.shape[0]):
    if (((df2["home_team"][i])==df2["toss_won"][i]) & ((df2["decision"][i])=="BAT FIRST") ):
        batting_team.append(df2["home_team"][i])
        bowling_team.append(df2["away_team"][i])
    elif (((df2["home_team"][i])==df2["toss_won"][i]) & ((df2["decision"][i])=="BOWL FIRST") ):
        batting_team.append(df2["away_team"][i])
        bowling_team.append(df2["home_team"][i])
    elif (((df2["away_team"][i])==df2["toss_won"][i]) & ((df2["decision"][i])=="BAT FIRST") ):
        batting_team.append(df2["away_team"][i])
        bowling_team.append(df2["home_team"][i])
    elif (((df2["away_team"][i])==df2["toss_won"][i]) & ((df2["decision"][i])=="BOWL FIRST") ):
        batting_team.append(df2["home_team"][i])
        bowling_team.append(df2["away_team"][i])
    else:
        print("wrong Data")

In [17]:
# Assigning list to our features
df2["batting_team"]=batting_team
df2["bowling_team"]=bowling_team

In [18]:
df=df.merge(df2[["match_id","batting_team","bowling_team"]])

In [19]:
df[["current_innings","innings_id","winner"]]

,current_innings,innings_id,winner
0,CSK,1,KKR
1,CSK,1,KKR
2,CSK,1,KKR
3,CSK,1,KKR
4,CSK,1,KKR
...,...,...,...
197861,RR,2,RR
197862,RR,2,RR
197863,RR,2,RR
197864,RR,2,RR


## Columns Requied:
batting_team <br>
bowling_team <br>
city <br>
runs_left <br>
balls_left <br>
wickets_left <br>
total_runs_ x<br>
crr <br>
rrr <br>
result

In [20]:
# Creating new dataframe with match_id,innings_id,runs features
total_runs_df=df.groupby(["match_id","innings_id"])["runs"].sum().reset_index()

In [21]:
# Creating total runs feature for both innings
total_runs_df1=total_runs_df[total_runs_df["innings_id"]==1] 
total_runs_df2=total_runs_df[total_runs_df["innings_id"]==2] 

In [22]:
total_runs_df1.rename(columns={"runs":"total_runs_x"},inplace=True)
total_runs_df2.rename(columns={"runs":"total_runs_y"},inplace=True)

In [23]:
match_df = df.merge(total_runs_df1[['match_id','total_runs_x']])
match_df = match_df.merge(total_runs_df2[['match_id','total_runs_y']])

In [24]:
match_df.drop(match_df[match_df["isRetiredHurt"]].index,inplace=True) # We dont want rows with any injured players for any abnormalities

In [25]:
match_df = match_df[match_df['innings_id'] == 2] # We are selecting only 2nd innings rows as we want to predict probability of winning of 2nd innings

In [27]:
match_df["current_score"]=match_df.groupby('match_id')['runs'].cumsum() # Creating new feature current_score from runs

In [38]:
match_df['runs_left'] = match_df ['total_runs_x'] - match_df ['current_score'] # Creating new feature runs_left from total runs and current score

In [29]:
match_df["balls_left"]=126 - (match_df['over']*6 + match_df['ball']) # Creating new feature balls_left from over and ball columns

In [37]:
# Creating wickets_left feature
match_df['wkt_batsman_name'] = match_df['wkt_batsman_name'].fillna("0")
match_df['wkt_batsman_name'] = match_df['wkt_batsman_name'].apply(lambda x:x if x == "0" else "1")
match_df['wkt_batsman_name'] = match_df['wkt_batsman_name'].astype('int')
wickets = match_df.groupby('match_id')['wkt_batsman_name'].cumsum()
match_df['wickets_left'] = 10 - wickets

In [39]:
# crr = runs/overs
match_df["crr"]=(match_df['current_score']*6)/ (120 - match_df['balls_left'])

In [40]:
#rrr= (6*runs_left)/balls_left
match_df['rrr'] = (match_df['runs_left']*6)/match_df['balls_left']

In [41]:
match_df = match_df[match_df['balls_left'] != 0] # As it would mess up rrr

In [43]:
match_df=match_df.merge(df2[["match_id","winner"]])

In [46]:
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

In [47]:
# Creating new feature result from batting_team and winner
match_df['result'] = match_df.apply(result,axis=1)

In [48]:
match_df=match_df.merge(df3[["match_id","city"]])

In [49]:
final_df = match_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets_left','total_runs_x','crr','rrr','result']]

In [51]:
# Splitting data
X = final_df.drop("result",axis=1)
y = final_df["result"]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

## Model Building

In [52]:
# Creating onehotencoder pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ohe = ColumnTransformer([
    ('ohe',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [54]:
pipe = Pipeline(steps=[
    ('step1',ohe),
    ('step2',LogisticRegression(solver='liblinear')),
])

In [55]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [56]:
y_pred = pipe.predict(X_test)

In [58]:
accuracy_score(y_test,y_pred)

0.788958848280796

In [60]:
pipe.predict_proba(X_test)

array([[9.99737370e-01, 2.62629877e-04],
       [1.82549968e-01, 8.17450032e-01],
       [6.73036036e-01, 3.26963964e-01],
       ...,
       [9.02435030e-01, 9.75649703e-02],
       [1.31034889e-01, 8.68965111e-01],
       [5.38262435e-01, 4.61737565e-01]])

In [96]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))